<a href="https://colab.research.google.com/github/nvvishnu/CS6910-Assignments/blob/main/Assignement1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
import pandas as pd
import numpy as np
import wandb
from keras.datasets import fashion_mnist
from matplotlib import pyplot

     |████████████████████████████████| 2.0MB 10.0MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 133kB 53.5MB/s 
     |████████████████████████████████| 163kB 51.9MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=42f94715318b78da0326e91c76c8450a626c921a8bd25207376121403a6eda76
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=92a5708117d21b166e44e5637c561e407a2736c4227fda6117053b247f4e3e90
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
np.unique(trainy)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [ ]:
trainy.T.shape

(60000,)

In [ ]:
np.exp(700)[0]

IndexError: ignored

In [ ]:

def softmax(arr):

    l = []

    flag = False

    cnt = 0

    y = []
    for i in arr : 
      if(i<=700):
         l.append(np.exp(i))
      else:
        flag = True
        cnt = cnt + 1
        y.append(i)

    if(flag):

       temp = np.zeros(shape=(len(arr),1),dtype=float)
       for it in y:
          temp[it][0] = 1.0/cnt;

       return temp   
         
    if(np.sum(l) != 0):
      l = np.array(l)
      l = (l)/np.sum(l)

      l = l.reshape((l.shape[0],1))

      return l

    else:
       return (1.0/len(arr))*(np.ones(shape=(len(arr),1),dtype=float))


def feed_forward(X,activation_function,weights,biases,output_function=softmax):

    X = X.flatten()
    num_hidden_layers = len(biases)
    # X is the input (x1,x2,...,xn)
    # activation_function is just the activation function s.t h_i = activation_function(a_i)
    # weights is a list of m*n where n is number of neurons in previous layer and m is number of neurons in current layer assume 1 based indexing 
    # assuming output function O is softmax function
    # biases is list of  [b1,b2,..,bk] for each layer

    X = X.reshape((X.shape[0],1))
    a = [X]
    h = [X]
    
    #I will add checks for the num_neurons
    for i in range(num_hidden_layers):
       if(i == 0):
         continue
       # a_i = W_ih_(i-1) + b_i
       a.append(np.matmul(weights[i],h[i-1])+biases[i])
       h.append(activation_function(a[len(a)-1])

    return [a,h,output_function(a[len(a)-1])]
    

SyntaxError: ignored

In [ ]:
#Helper functions
def sigmoid(z):
  return 1/(1+exp(-z))
def tanh(z):
  return (exp(z)-exp(-z))/(e(z)+e(-z))
def sigmoid_grad(z):
  return sigmoid(z)/(1-sigmoid(z))
def tanh_grad(z):
  return 1-(tanh(z))*(tanh(z))
def relu(z):
  return max(0,z)
def relu_grad(z):
  if(z<0):
    return 0
  else:
    return 1
def grad_output(l,y):
  n=y.size()[0]
  e=np.zeroes(n)
  for i in range(0,n):
    if(l==(i+1)):
      e[i]=1
  return -(e-y)

In [ ]:
def caluculate_gradients(y_true,y_out,h,a,weights,act_fn_derivative):
    
    #delta(A_l) = -(e_ytrue - y_out)
    #delta(h_i) = (W_i+1).Tdelta(A_i+1)  (for i = L-1 to 1)
    #delta(A_i) = delta(h_i)*(activation_function'(A_i)) for(i = L-1 to 1)
    #delta(W_k) = delta(A_k)(h_(k-1).T) for(i = L to i = 1)
    #delta(b_k) = delta(A_k)


    e = np.zeros(len(y_out))
    e[y_true] = 1
    A = y_out-e
    l = len(y_out)
    delta_W = []
    delta_W.append(np.matmul(A,h[l-1].T))
    delta_B.append(A)
    l = l-1
    while(l >= 1):

       H = np.matmul(weights[l+1].T,A)
       A = H*(act_fn_derivative(a[l]))     # assuming a is 1 based index 
       delta_W.append(np.matmul(A,h[l-1].T))
       delta_B.append(A)
       l = l-1

    delta_W.reverse()
    delta_B.reverse()
    return [delta_W,delta_B]

    



In [ ]:
def upd_lst(lst1,lst2,operation,eeta):
   l = []

   for (x,y) in zip(lst1,lst2):
     if(operation == '+'):
       l.append(x+(eeta*y))
     else:
       l.append(x-(eeta*y))

   return l

In [3]:
def sgd(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative):

     for i in range(num_epochs): 
          cnt = 0
          cum_W = []
          cum_B = []
          for j in range(X.shape[0]):

             cnt = cnt + 1
             r = feed_forward(X[j],activation_function,weights,biases)
             a = r[0]
             h = r[1]
             yout = r[2]
             gradients = caluculate_gradients(Y[j],yout,h,a,weights)
             cum_W = upd_lst(cum_W,gradients[0],'+',1)
             cum_B = upd_lst(cum_B,gradients[1],'+',1)
             if(cnt == batch_size): 
                weights = upd_lst(weights,cum_W,'-',(learning_rate*(1-weight_decay))/batch_size)
                biases = upd_lst(biases,cum_B,'-',(learning_rate*(1-weight_decay))/batch_size)
                cum_W = []
                cum_B = []
                cnt = 0
      
     return [weights,biases]

In [4]:
def backpropagation(X,Y,num_epochs,num_hidden_layers,sz_hidden_layer,weight_decay,learning_rate,optimizer,batch_size,weight_initialisation,activation_function):
    
    
    #do the weights Initialisation here based on weight_initialisation,num_hidden_layers,sz_each_hidden_layer
    
    weights = [0]  #dummy value
    biases =  [0]  #dummy value

    if(weight_initialisation == 'random'):
          
          num_classes = len(np.unique(Y))
          i = len(X[0].flatten())
          a = -100
          b = 100

          for j in range(num_hidden_layers-1):
              weights.append((200)*np.random.rand(sz_hidden_layer,i)-100)
              biases.append((200)*np.random.rand(sz_hidden_layer)-100)
              i = sz_hidden_layer

          weights.append((200)*np.random.rand(num_classes,i)-100)
          biases.append((200)*np.random.rand(num_classess)-100)
          
    else:
         print("Please Implement Xavier haha !!")  
    if(activation_function=='sigmoid'):
      activation_function=sigmoid
      act_fn_derivative = sigmoid_grad
    elif(activation_function=='tanh'):
      activation_function=tanh
      act_fn_derivative = tanh_grad
    elif(activation_function=='ReLU'):
      activation_function= relu
      act_fn_derivative = relu_grad
        

    if(optimizer == 'sgd'):
       return sgd(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)
      
    if(optimizer == 'momentum'): 
       return momentum(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)

    if(optimizer == 'nesterov'): 
       return nesterov(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)

    if(optimizer == 'rmsprop'):
       return rmsprop(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)

    if(optimizer == 'adam'):
      return adam(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)

    if(optimizer == 'nadam'):
      return nadam(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)
  


In [2]:
def momentum(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,gamma,act_fn_derivative):
  #w_ahead=w_{t} - gamma. update_{t-1}
  #update{t} = gamma . update_{t-1} + learning_rate * grad(w_lookahead)

  #Initialisation of weights_grad_tot, biases_grad_tot
  weights_grad_tot=[]
  for w in weights:
    weights_grad_tot.append(np.zeroes(w.size()))
  biases_grad_tot=[]
  for b in biases:
    biases_grad_tot.append(np.zeroes(b.size()))

  #Initialisation of update_weights
  update_weights=weights_grad_tot
  update_biases=biases_grad_tot

  for i in range(num_epochs):
    num_hidden_layers=biases.size()[0] #Biases is a list of 1D arrays. Thus, the number of hidden layers is given by the size of this list
    Y_pred=feed_forward(X,activation_function,weights,biases) #Find prediction for each data instance of the data set
    num_data_point_done=0
    for x,y,y_pred in zip(X,Y,Y_pred):    
      [weights_grad,biases_grad] =  calculate_gradient(y,y_pred,h,a,weights,act_fn_derivative)
      weights_grad_tot = weights_grad_tot+weights_grad
      biases_grad_tot = biases_grad_tot + biases_grad      
      num_data_point_done=num_data_point_done+1
      if(num_data_point_done==batch_size):
        update_weights = update_weights + gamma*update_weights + learning_rate * weights_grad_tot
        update_biases = update_biases + gamma*update_weight + learning_rate * biases_grad_tot
        weights=weights-update_weights
        biases=biases - update_biases
        num_data_point_done=0
  
def nesterov(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,gamma,act_fn_derivative):
  #w_ahead=w_{t} - gamma. update_{t-1}
  #update{t} = gamma . update_{t-1} + learning_rate * grad(w_lookahead)
  
  #Initialisation of weights_grad_tot, biases_grad_tot
  weights_grad_tot=[]
  for w in weights:
    weights_grad_tot.append(np.zeroes(w.size()))
  biases_grad_tot=[]
  for b in biases:
    biases_grad_tot.append(np.zeroes(b.size()))
  #Initial value of w_head
  w_ahead = weights

  #Initialisation of update_weights
  update_weights=weights_grad_tot
  update_biases=biases_grad_tot

  for i in range(num_epochs):
    num_hidden_layers=biases.size()[0] #Biases is a list of 1D arrays. Thus, the number of hidden layers is given by the size of this list
    Y_pred=feed_forward(X,activation_function,weights,biases) #Find prediction for each data instance of the data set
    num_data_point_done=0
    for x,y,y_pred in zip(X,Y,Y_pred):          
      [weights_grad,biases_grad] =  calculate_gradient(y,y_pred,h,a,w_ahead,act_fn_derivative)
      weights_grad_tot = weights_grad_tot+weights_grad
      biases_grad_tot = biases_grad_tot + biases_grad             
      num_data_point_done=num_data_point_done+1
      if(num_data_point_done==batch_size):
        w_ahead = weights -  update_weights
        b_ahead = biases -  update_biases
        update_weights = update_weights + gamma*update_weights + learning_rate * weights_grad_tot
        update_biases = update_biases + gamma*update_weight + learning_rate * biases_grad_tot    
        weights=weights-update_weights
        biases=biases - update_biases
        num_data_point_done=0

  
def adagrad(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,gamma,act_fn_derivative):
  #w_ahead=w_{t} - gamma. update_{t-1}
  #update{t} = gamma . update_{t-1} + learning_rate * grad(w_lookahead)
  # w_ahead= weights
  # b_ahead=biases
  for i in range(num_epochs):
    num_hidden_layers=biases.size()[0] #Biases is a list of 1D arrays. Thus, the number of hidden layers is given by the size of this list
    Y_pred=feed_forward(X,activation_function,weights,biases) #Find prediction for each data instance of the data set
    weights_grad_tot=[]
    for w in weights:
      weights_grad_tot.append(np.zeroes(w.size()))
    biases_grad_tot=[]
    for b in biases:
      biases_grad_tot.append(np.zeroes(b.size()))
    num_data_point_done=0
    v_weight = 0 #Intitial value of 0
    v_bias=0
    epsilon = 0 #Figure out what is epsilon
    for x,y,y_pred in zip(X,Y,Y_pred):    
      [weights_grad,biases_grad] =  calculate_gradient(y,y_pred,h,a,weights,act_fn_derivative)
      weights_grad_tot = weights_grad_tot+weights_grad
      biases_grad_tot = biases_grad_tot + biases_grad
      num_data_point_done=num_data_point_done+1
      if(num_data_point_done==batch_size):
        v_weight = v_weight + (weights_grad_total)**2
        v_bias = v_bias + (biases_grad_total)**2
        v_weight_dum = learning_rate/np.sqrt(v_weight+epsilon)
        v_bias_dum = learning_rate/np.sqrt(v_bias+epsilon)
        weights=weights- v_weight_dum * weights_grad_tot
        biases=biases - v_bias_dum * bias_grad_tot
        num_data_point_done=0
    
def rmsprop(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,gamma,act_fn_derivative):
  #w_ahead=w_{t} - gamma. update_{t-1}
  #update{t} = gamma . update_{t-1} + learning_rate * grad(w_lookahead)
  # w_ahead= weights
  # b_ahead=biases
  for i in range(num_epochs):
    num_hidden_layers=biases.size()[0] #Biases is a list of 1D arrays. Thus, the number of hidden layers is given by the size of this list
    Y_pred=feed_forward(X,activation_function,weights,biases) #Find prediction for each data instance of the data set
    weights_grad_tot=[]
    for w in weights:
      weights_grad_tot.append(np.zeroes(w.size()))
    biases_grad_tot=[]
    for b in biases:
      biases_grad_tot.append(np.zeroes(b.size()))
    num_data_point_done=0
    v_weight = 0 #Intitial value of 0
    v_bias=0
    epsilon = 0 #Figure out what is epsilon
    beta=0
    for x,y,y_pred in zip(X,Y,Y_pred):    
      [weights_grad,biases_grad] =  calculate_gradient(y,y_pred,h,a,weights,act_fn_derivative)
      weights_grad_tot = weights_grad_tot+weights_grad
      biases_grad_tot = biases_grad_tot + biases_grad
      num_data_point_done=num_data_point_done+1
      if(num_data_point_done==batch_size):
        v_weight = beta*v_weight + (1-beta)*(weights_grad_total)**2
        v_bias = beta*v_bias + (1-beta)*(biases_grad_total)**2
        v_weight_dum = learning_rate/np.sqrt(v_weight+epsilon)
        v_bias_dum = learning_rate/np.sqrt(v_bias+epsilon)
        weights=weights- v_weight_dum * weights_grad_tot
        biases=biases - v_bias_dum * bias_grad_tot
        num_data_point_done=0
      
      
  


